In [1]:
# Copyright (c) 2020-2021, NVIDIA CORPORATION.  All rights reserved.
#
# NVIDIA CORPORATION and its licensors retain all intellectual property
# and proprietary rights in and to this software, related documentation
# and any modifications thereto.  Any use, reproduction, disclosure or
# distribution of this software and related documentation without an express
# license agreement from NVIDIA CORPORATION is strictly prohibited.


In [1]:
import getpass

user = getpass.getuser()
from omni.isaac.kit import SimulationApp

# Set the path below to your desired nucleus server
# Make sure you installed a local nucleus server before this
try:
    simulation_app.close()
except NameError:
    simulation_app = SimulationApp({"livesync_usd": f"omniverse://localhost/Users/{user}/temp_jupyter_stage.usd"})


Starting kit application with the fillowing args:  ['/home/ksterx/.local/share/ov/pkg/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/ksterx/.local/share/ov/pkg/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/ksterx/.local/share/ov/pkg/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/ksterx/.local/share/ov/pkg/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/ksterx/.local/share/ov/pkg/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/ksterx/.local/share/jupyter/runtime/kernel-6991d523-6704-

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA GeForce RTX 3060
   Kernel cache: /home/ksterx/.cache/warp/0.2.2
[1.449s] [ext: omni.kit.menu.create-1.0.2] startup
[1.450s] [ext: omni.mdl-0.1.0] startup
[1.509s] [ext: omni.kit.window.file_importer-1.0.4] startup
[1.510s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[1.511s] [ext: omni.kit.material.library-1.3.10] startup
[1.513s] [ext: omni.kit.window.drop_support-1.0.0] startup
[1.514s] [ext: omni.kit.window.file-1.3.16] startup
[1.515s] [ext: omni.kit.context_menu-1.3.9] startup
[1.518s] [ext: omni.kit.window.property-1.6.3] startup
[1.519s] [ext: omni.kit.window.content_browser-2.4.28] startup
[1.526s] [ext: omni.kit.widget.stage-2.6.15] startup
[1.529s] [ext: omni.isaac.version-1.0.0] startup
[1.530s] [ext: omni.kit.property.usd-3.14.9] startup
[1.553s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[1.556s] [ext: omni.hydra.rtx-0.1.0] startup
[1.563s] [ext: omni.renderer-rtx-0.0.0] startup
[1.563s] [ext: omn

In [2]:
from omni.isaac.core import World
from omni.isaac.core.objects import DynamicCuboid
import numpy as np


In [3]:
world = World(stage_units_in_meters=1.0)
world.scene.add_default_ground_plane()
# A render/ step or an update call is needed to reflect the changes to the opened USD in Isaac Sim GUI
#  Note: avoid pressing play/ pause or stop in the GUI in this workflow.
world.render()


In [4]:
stage = world.scene.stage

In [5]:
from pxr import Gf, PhysxSchema, Sdf, Usd, UsdGeom, UsdPhysics, UsdShade
from omni.physx.scripts import deformableUtils, physicsUtils, utils
prim_path = "/World/DeformableBody"
usd_path = "omniverse://localhost/Users/ksterx/cube.usd"
pos = Gf.Vec3f(2.0, 2.0, 2.0)
size = Gf.Vec3f(0.05, 0.05, 0.05)
orient = Gf.Quatf(1.0)


In [6]:
def create(stage, path, name, position, size, mesh_path, phys_mat_path, grfx_mat_path):
    cube_path = path.AppendChild(name)
    stage.DefinePrim(cube_path).GetReferences().AddReference(mesh_path)
    skinMesh = UsdGeom.Mesh.Define(stage, cube_path)
    skinMesh.AddTranslateOp().Set(position)
    skinMesh.AddOrientOp().Set(Gf.Quatf(1.0))
    skinMesh.AddScaleOp().Set(Gf.Vec3f(size, size, size))
    deformableUtils.add_physx_deformable_body(
        stage,
        cube_path,
        simulation_hexahedral_resolution=3,
        collision_simplification=True,
        self_collision=False,
        solver_position_iteration_count=pos_iterations,
    )
    physicsUtils.add_physics_material_to_prim(stage, skinMesh.GetPrim(), phys_material_path)
    physxCollisionAPI = PhysxSchema.PhysxCollisionAPI.Apply(skinMesh.GetPrim())
    physxCollisionAPI.GetContactOffsetAttr().Set(0.02)
    physxCollisionAPI.CreateRestOffsetAttr().Set(0.001)
    omni.kit.commands.execute(
        "BindMaterialCommand", prim_path=cube_path, material_path=grfx_material, strength=None
    )

In [8]:
parent = stage.GetPrimAtPath("/World").GetPath()
env_xform = UsdGeom.Xform.Define(stage, parent.AppendChild("Deformables"))
env_xform.AddTranslateOp().Set(Gf.Vec3f(0.0, 0.0, 0.0))

True

In [9]:
from pxr import Gf, PhysxSchema, Sdf, Usd, UsdGeom, UsdPhysics, UsdShade
from omni.physx.scripts import deformableUtils, utils, physicsUtils
import omni
phys_scene = UsdPhysics.Scene.Define(stage, parent.AppendChild("physicsScene"))
phys_scene.CreateGravityDirectionAttr().Set(Gf.Vec3f(0.0, 0.0, -1.0))
phys_scene.CreateGravityMagnitudeAttr().Set(9.807)
utils.set_physics_scene_asyncsimrender(phys_scene.GetPrim())
physxSceneAPI = PhysxSchema.PhysxSceneAPI.Apply(phys_scene.GetPrim())
physxSceneAPI.CreateFrictionOffsetThresholdAttr().Set(0.001)
physxSceneAPI.CreateFrictionCorrelationDistanceAttr().Set(0.0005)
physxSceneAPI.CreateGpuTotalAggregatePairsCapacityAttr().Set(10 * 1024)
physxSceneAPI.CreateGpuFoundLostPairsCapacityAttr().Set(10 * 1024)
physxSceneAPI.CreateGpuCollisionStackSizeAttr().Set(64 * 1024 * 1024)
physxSceneAPI.CreateGpuFoundLostAggregatePairsCapacityAttr().Set(10 * 1024)
# clamp iterations:
pos_iterations = 20
vel_iterations = 1
physxSceneAPI.GetMaxIterationCountAttr().Set(pos_iterations)
UsdGeom.SetStageUpAxis(stage, UsdGeom.Tokens.z)
UsdGeom.SetStageMetersPerUnit(stage, 1.0)

# setup ground collision plane:
utils.addPlaneCollider(stage, "/World/physicsGroundPlaneCollider", "Z")

# deformable material path
deformable_material_path = omni.usd.get_stage_next_free_path(stage, "/DeformableBodyMaterial", True)
deformableUtils.add_deformable_body_material(
    stage,
    deformable_material_path,
    youngs_modulus=10000000.0,
    poissons_ratio=0.499,
    damping_scale=0.0,
    elasticity_damping=0.0001,
    dynamic_friction=1.0,
    density=300,
)
deformable_material2_path = omni.usd.get_stage_next_free_path(stage, "/DeformableBodyMaterial2", True)
deformableUtils.add_deformable_body_material(
    stage,
    deformable_material2_path,
    youngs_modulus=4000000.0,
    poissons_ratio=0.499,
    damping_scale=0.0,
    elasticity_damping=0.005,
    dynamic_friction=0.05,
    density=100,
)

True

In [10]:
demo_camera = Sdf.Path("/World/Camera")
cam = UsdGeom.Camera.Define(stage, demo_camera)

In [11]:
world.render()

In [ ]:
grfx_mat_path = 

In [ ]:
create(stage, prim_path, "cube", pos, size, usd_path, deformable_material_path, grfx_mat_path)

In [ ]:
world.reset()
for i in range(500):
    position, orientation = fancy_cube.get_world_pose()
    linear_velocity = fancy_cube.get_linear_velocity()
    print("Cube position is : " + str(position))
    print("Cube's orientation is : " + str(orientation))
    print("Cube's linear velocity is : " + str(linear_velocity))
    # we have control over stepping physics and rendering in this workflow
    # things run in sync
    world.step(render=True)  # execute one physics step and one rendering step


In [ ]:
# Cleanup application
simulation_app.close()
